In [1]:
##indexing##
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
loader=WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/"),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        ),
    )
)

docs=loader.load()

#split
text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300,chunk_overlap=50)
splits=text_splitter.split_documents(docs) #->list

#embed
embedding_model=HuggingFaceEmbeddings(model_name="BAAI/bge-small-zh-v1.5")
vectorstore=Chroma.from_documents(documents=splits,embedding=embedding_model)
retriever=vectorstore.as_retriever()

#ollama llm
llm=ChatOllama(model="llama3.2:3b-instruct-q8_0",temperature=0)

c:\Users\user\anaconda3\envs\torch-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate,ChatPromptTemplate


examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jerry was born in what country?",
        "output": "what is Jerry personal history?",
    },
]

examples_prompt=ChatPromptTemplate.from_messages(
    [
        ("human","{input}"),
        ("ai","{output}"),
    ]
)

few_shot_prompt=FewShotChatMessagePromptTemplate(
    example_prompt=examples_prompt,
    examples=examples
)

prompt_step_back=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge.
            Your task is to step back and paraphrase a question to a more generic step-back question,
            which is easier to answer. Here are a few examples:""",
        ),

        few_shot_prompt,

        (
            "user","{question}"
        ),
    ]
)



In [ ]:
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

generate_queries_step_back=prompt_step_back | llm | StrOutputParser()

question="What is task decomposition for LLM agents"

normal_chain=(
    itemgetter("question")
    | retriever
)

In [19]:

response_prompt_template = """
You are an expert of world knowledge. I am going to ask you a question.
Your response should be comprehensive and not contradicted with the following context
if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
respones_prompt=ChatPromptTemplate.from_template(response_prompt_template)


final_rag_chain=(
    {
        "normal_context":normal_chain,
        "step_back_context": generate_queries_step_back | retriever,
        "question":itemgetter("question")
    }
    | respones_prompt
    | llm
    | StrOutputParser()

)
final_rag_chain.invoke({"question": question})

'Task decomposition is a crucial component of planning in LLM-powered autonomous agent systems, where the large language model (LLM) breaks down complex tasks into smaller, manageable subgoals. This process enables efficient handling of complex tasks and allows the agent to plan ahead.\n\nThere are three ways task decomposition can be done:\n\n1. **Simple prompting**: The LLM is instructed with simple prompts like "Steps for XYZ. 1." or "What are the subgoals for achieving XYZ?" to decompose the task into smaller steps.\n2. **Task-specific instructions**: Task-specific instructions, such as "Write a story outline" for writing a novel, can be used to guide the LLM in decomposing the task.\n3. **Human inputs**: Human inputs can also be used to provide guidance on how to decompose the task.\n\nTwo techniques that extend traditional task decomposition are:\n\n1. **Chain of Thought (CoT)**: CoT is a standard prompting technique for enhancing model performance on complex tasks. The model is 